# Seat availability at La Permanence: a first look at the data

[La Permanence](https://www.la-permanence.com) is a coworking space with two locations in Paris (*rue du Fer à Moulin* and *rue d'Alésia*).  Our purpose in this repository is to model and then predict the number the available seats, and this will be done in separate notebooks.  In this notebook, our goal is simply to familiarize ourselves with the data.  The steps taken in this notebook will be performed as a preliminary to most other notebooks, without explanation.

## Imports

In [1]:
import datetime
import os
import numpy as np
import pandas as pd
import pytz
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
%matplotlib inline

## Constants

In [2]:
# Timezones
tz_utc = pytz.timezone("UTC")  # timestamp is in UTC standard
tz_paris = pytz.timezone("Europe/Paris")  # locations are in Paris, France

# Resampling rule: resolution of time regularization
RESOL = 2  # 10
RULE = f"{RESOL}T"
# Duration of validation data
ONE_HOUR = 60 // RESOL
ONE_DAY = 24*ONE_HOUR
ONE_WEEK = 7*ONE_DAY
TWO_WEEKS = 2*ONE_WEEK
days_of_the_week = {0: 'Monday',
                    1: 'Tuesday', 
                    2: 'Wednesday',
                    3: 'Thursday',
                    4: 'Friday',
                    5: 'Saturday', 
                    6: 'Sunday'}

# Plotting
SAVEFIGS = False
FIGSIZE = (16, 6)
ALPHA = 0.7
# Colors 
COL_ALESIA = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
COL_MOULIN = plt.rcParams['axes.prop_cycle'].by_key()['color'][1]
COL_TRAIN = plt.rcParams['axes.prop_cycle'].by_key()['color'][2]
COL_VALID = plt.rcParams['axes.prop_cycle'].by_key()['color'][3]
COL_PREDICT = plt.rcParams['axes.prop_cycle'].by_key()['color'][4]
COL_AVG = plt.rcParams['axes.prop_cycle'].by_key()['color'][5]
COL_MED = plt.rcParams['axes.prop_cycle'].by_key()['color'][6]

## Functions

In [3]:
def split_vals(df, n):
    return df[:n].copy(), df[n:].copy()


def new_y_ticklabel(text):
    """Reformat text for y ticklabels in heatmaps"""
    ts = pd.to_datetime(text.get_text()).tz_localize(tz_utc).tz_convert(tz_paris)
    return ts.strftime('%d %b %Y | %a') if ts.weekday() == 0 else ts.strftime('%a')

# Data: loading and preparation
The dataset is saved in a `csv` file located in `~/Projects/la_permanence/attendance.csv`.  

**Note**: `attendance.csv` is a misnomer, as the file actually records the number of *available* seats, rather than the number of seats used.  

## Metadata on the dataset

In [4]:
filename = 'attendance.csv'
foldername = 'Projects/la_permanence'
filepath = os.path.join(os.path.expanduser("~"), foldername, filename)
filesize = os.path.getsize(filepath)

COL_WIDTH = 20  
# for display in this cell

if filesize > 1e6:
    print("{0}{1:d}MB".format("File size:".ljust(COL_WIDTH), filesize//int(1e6)))
elif filesize > 1e3:
    print("{0}{1:d}KB".format("File size:".ljust(COL_WIDTH), filesize//int(1e3)))
else:
    print("{0}{1:d}B".format("File size:".ljust(COL_WIDTH), filesize))

line_number = !wc -l < {filepath}
line_number = int(line_number[0])
print("{0}{1}".format("Number of rows:".ljust(COL_WIDTH), line_number))
first_row = !head -1 {filepath}
column_names = first_row[0].split(',')
print("{0}{1}".format("Names of columns:".ljust(COL_WIDTH), ", ".join(column_names)))

File size:          1MB
Number of rows:     65844
Names of columns:   timestamp, Moulin, Alésia


## More details on the columns
- `timestamp` gives the date and time (UTC) of data collection in the format `YYYY-MM-DD-hh-mm-ss` where 
  - `YYYY=`year,
  - `MM=`month,
  - `DD=`day,
  - `hh=`hour,
  - `mm=`minute,
  - `ss`=second;
- `Moulin` gives the number of available seats at the *rue du Fer à Moulin* location (maximum is 63);
- `Alésia` gives the number of available seats at the *rue d'Alésia* location (maximum is 82).  

## Load the data

In [5]:
%time
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d-%H-%M-%S')

raw_data = pd.read_csv(
    filepath,
    sep=',',
    dtype={'Moulin': np.uint8, 'Alésia': np.uint8},
    parse_dates=['timestamp'],
    date_parser=dateparse
)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 10 µs


ValueError: cannot safely convert passed user dtype of uint8 for float64 dtyped data in column 1

## Adjusting for timezones

The `timestamp` values are in the UTC standard, while the coworking spaces are located in Paris, France:

In [ ]:
%time
raw_data['timestamp'] = raw_data['timestamp'].apply(lambda ts: ts.tz_localize(tz_utc).tz_convert(tz_paris))

## Time regularization

The data was collected at irregular times, as can be seen by zooming in on a short window frame:

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
idxs = range(100, 200)
window = raw_data.loc[idxs, raw_data.columns]
the_day = window.loc[window.index.min(), 'timestamp']
the_hours = list(set([x.hour for x in  window.loc[idxs, 'timestamp']]))

hour_range = pd.date_range(pd.Timestamp(the_day.year, the_day.month, the_day.day, the_day.hour), periods=the_hours[-1] - the_hours[0], freq= 'H')

window.set_index('timestamp').plot(style='-o', **{"markersize": 3, "markerfacecolor": "k", "markeredgecolor": "k"}, alpha=ALPHA, ax=ax);

ax.set_title("Seat availability at La Permanence coworking spaces", fontsize=18);

ax.set_ylabel("Number of available seats", fontsize=14);
ax.set_yticks(range(0, 80, 10));

ax.set_xlabel(the_day.strftime('%A %d %B %Y'), fontsize=14);
ax.set_xticks(hour_range);
ax.set_xticklabels([x.strftime("%H:00") for x in hour_range]);

ax.legend(["rue du Fer à Moulin", "rue d'Alésia"]);
ax.grid();

Thus we resample (at a resolution specified in `RULE`) and interpolate to regularize the data:

In [ ]:
raw_data.set_index("timestamp", inplace=True)
raw_data = raw_data.resample(RULE).mean().interpolate().round().astype(np.uint8)
raw_data.reset_index('timestamp', inplace=True)

## Additional datetime features
For future use we extract now additional features from the `timestamp`.

In [ ]:
%time 
raw_data['Date'] = raw_data['timestamp'].apply(
    lambda ts: pd.Timestamp(ts.year, ts.month, ts.day).tz_localize(tz_paris)
)

attributes = ['Dayofweek', 'Hour', 'Minute', 'Week']
for attr in attributes:
    raw_data[attr] = getattr(raw_data['timestamp'].dt, attr.lower())
raw_data['Minuteofday'] = 60*raw_data['Hour'] + raw_data['Minute']
raw_data['Minuteofweek'] = 24*60*raw_data['Dayofweek'] + raw_data['Minuteofday']
print("\nColumn names:\n\n{0}".format("\n".join(raw_data.columns)))

## Training and validation split
We wil
l shortly derive average values of the data, which can be used to make forecasts.  Thus we will work on a training dataset obtained by removing the last week's worth of data:

In [ ]:
n_valid = ONE_WEEK
n_trn = raw_data.shape[0] - n_valid
train_data, valid_data = split_vals(raw_data, n_trn)
train_timestamps, valid_timestamps = split_vals(raw_data[['timestamp']], n_trn)

## Visualization of the timeseries

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4));

plt.subplots_adjust(hspace=0.6)

ts_min = raw_data['timestamp'].min()
ts_max = raw_data['timestamp'].max()
the_date_range = pd.date_range(ts_min.date(), ts_max.date());

# rue du Fer à Moulin
raw_data.plot(x='timestamp', y='Moulin', color=COL_MOULIN, alpha=ALPHA, ax=ax);

ax.set_title("Seat availability (rue du Fer à Moulin)", fontsize=18)

ax.set_ylabel("Available seats", fontsize=14);
ax.set_yticks(range(0, 80, 10))

ax.axes.get_xaxis().label.set_visible(False);
ax.set_xticks(the_date_range);
ax.set_xticklabels([pd.to_datetime(x).strftime("%a %d %b")  #  %d %b %Y | %A
                     if x.weekday() in [0]
                     else pd.to_datetime(x).strftime("%a")
                     for x in the_date_range])
ax.tick_params(axis='x', labelrotation=60)

ax.get_legend().remove();
ax.grid();
if SAVEFIGS:
    plt.savefig("moulin-availability-timeseries.png");

# Averaging

Now we derive a representative for each day of the week.  

## Averages by day

In [ ]:
moulin = train_data.set_index('timestamp').drop('Alésia', axis=1)
moulin_by_day_of_week = pd.pivot_table(moulin, index=moulin.index.hour, columns=moulin.index.dayofweek, values='Moulin', aggfunc='mean')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

plt.subplots_adjust(hspace=0.4)

moulin_by_day_of_week.plot(alpha=ALPHA, ax=ax);

ax.set_title("Average seat availability (rue du Fer à Moulin)", fontsize=18)
ax.set_ylabel("Available seats", fontsize=14)
ax.set_xlabel("Time of day", fontsize=14)

ax.set_yticks(range(0, 80, 10))
ax.set_xticks(range(0,24, 3))
ax.set_xticklabels(["{:02d}:00".format(x) for x in ax.get_xticks()])

ax.legend(days_of_the_week.values())
ax.grid()
if SAVEFIGS:
    plt.savefig("moulin-average-by-day.png")


We can see that the different days of the week have distinct average behaviour.  Is there a lot of variation about these representatives?  Can we have a sense of the distribution of the seat availability over the different weeks of data?

# Aggregate plots

We will now superpose plots of the data over entire days and weeks.  To this end we generate pivot tables.

## Daily view
Rearrange the (entire) dataset, one day per row:

In [ ]:
daily_pivot = pd.pivot_table(raw_data, index='Date', columns='Minuteofday', values='Moulin')

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE);

colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][0:7]
ax.set_prop_cycle('color', colors)
daily_pivot.transpose().plot(style='-', linewidth=4, alpha=0.3, ax=ax);

ax.set_title("Seat availability: daily view (rue du Fer à Moulin)", fontsize=18);

ax.set_xticks([h*ONE_HOUR*RESOL for h in range(0, 24)])
ax.set_xticklabels(["{0:02d}:00".format(x//ONE_HOUR//RESOL) for x in ax.get_xticks()])
ax.set_xlabel("Time of day", fontsize=14);

ax.set_ylabel("Available seats", fontsize=14);
ax.set_yticks(range(0, 80, 10));


ax.legend(days_of_the_week.values(), loc='lower left');
ax.grid();
if SAVEFIGS:
    plt.savefig('moulin-daily-availability.png')

It is quite difficult to get a sense of what is going on.  Things are a bit more instructive when looking at entire weeks.

## Weekly view
Rearrange the (training and validation) datasets, one week per row:

In [ ]:
weekly_train_pivot = pd.pivot_table(train_data, index='Week', columns='Minuteofweek', values='Moulin')
weekly_valid_pivot = pd.pivot_table(valid_data, index='Week', columns='Minuteofweek', values='Moulin')

In [ ]:
weekly_valid_pivot

Calculate weekly averages over training set:

In [ ]:
weekly_train_avg = train_data.groupby('Minuteofweek')[['Moulin']].mean().round().astype(np.int8).reset_index()

In the following, we plot the training data (in green), its average (in black), and the data for the last week of the validation set (in red):

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True);

custom_lines = [Line2D([0], [0], color=COL_TRAIN, lw=4),
                Line2D([0], [0], color=COL_VALID, lw=4),
                Line2D([0], [0], color=COL_AVG, lw=4)]

ax1.set_prop_cycle('color', colors)

weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax1);
if weekly_valid_pivot.shape[0] > 1:
    weekly_valid_pivot.iloc[-2].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax1);
ax1.plot(weekly_train_avg['Minuteofweek'], weekly_train_avg['Moulin'], COL_AVG, alpha=ALPHA);

ax1.set_title("Seat availability: weekly view (rue du Fer à Moulin)", fontsize=18);

ax1.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax1.set_xticklabels(days_of_the_week.values());
ax1.set_xlabel("Day of week", fontsize=14);
ax1.tick_params(axis='x', labelrotation=90);

ax1.set_ylabel("Available seats", fontsize=14);
ax1.set_yticks(range(0, 80, 10));

# ax1.get_legend().remove();
# ax1.legend(['])
ax1.legend(custom_lines, ['training', 'validation', 'average'], loc='upper left');
ax1.grid();

ax2.set_prop_cycle('color', colors)

weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax2);
weekly_valid_pivot.iloc[-1].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax2);
ax2.plot(weekly_train_avg['Minuteofweek'], weekly_train_avg['Moulin'], color=COL_AVG, alpha=ALPHA);

ax2.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax2.set_xticklabels(days_of_the_week.values());
ax2.set_xlabel("Day of week", fontsize=14);
ax2.tick_params(axis='x', labelrotation=90);

ax2.set_ylabel("Available seats", fontsize=14);
ax2.set_yticks(range(0, 80, 10));

ax2.legend(custom_lines, ['training', 'validation', 'average'], loc='upper right');
ax2.grid();

if SAVEFIGS:
    plt.savefig('moulin-weekly-timeseries.png')

This plot gives some visual evidence that the training data (in green) is well represented by its average (in black).  The deviation seems to be due to a few outlying values (some of which are actually interpolated values replacing missing data).  The validation set turns out to be quite distinct from the previous weeks, and it is not too surprising that the average does not predict it very well.

# Using the median

In [ ]:
weekly_train_med = train_data.groupby('Minuteofweek')[['Moulin']].median().round().astype(np.int8).reset_index()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True);

custom_lines = [Line2D([0], [0], color=COL_TRAIN, lw=4),
                Line2D([0], [0], color=COL_VALID, lw=4),
                Line2D([0], [0], color=COL_MED, lw=4)]

ax1.set_prop_cycle('color', colors)

weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax1);
if weekly_valid_pivot.shape[0] > 1:
    weekly_valid_pivot.iloc[-2].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax1);
ax1.plot(weekly_train_med['Minuteofweek'], weekly_train_med['Moulin'], color=COL_MED, alpha=ALPHA);

ax1.set_title("Seat availability: weekly view (rue du Fer à Moulin)", fontsize=18);

ax1.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax1.set_xticklabels(days_of_the_week.values());
ax1.set_xlabel("Day of week", fontsize=14);
ax1.tick_params(axis='x', labelrotation=90);

ax1.set_ylabel("Available seats", fontsize=14);
ax1.set_yticks(range(0, 80, 10));

# ax1.get_legend().remove();
# ax1.legend(['])
ax1.legend(custom_lines, ['training', 'validation', 'median'], loc='upper left');
ax1.grid();

ax2.set_prop_cycle('color', colors)

weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax2);
weekly_valid_pivot.iloc[-1].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax2);
ax2.plot(weekly_train_med['Minuteofweek'], weekly_train_med['Moulin'], color=COL_MED, alpha=ALPHA);

ax2.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax2.set_xticklabels(days_of_the_week.values());
ax2.set_xlabel("Day of week", fontsize=14);
ax2.tick_params(axis='x', labelrotation=90);

ax2.set_ylabel("Available seats", fontsize=14);
ax2.set_yticks(range(0, 80, 10));

ax2.legend(custom_lines, ['training', 'validation', 'average'], loc='upper right');
ax2.grid();

if SAVEFIGS:
    plt.savefig('moulin-weekly-timeseries.png')

# Comparing median and average

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True);

custom_lines = [ 
    #Line2D([0], [0], color=COL_TRAIN, lw=4),
    Line2D([0], [0], color=COL_VALID, lw=4),
    Line2D([0], [0], color=COL_AVG, lw=4),
    Line2D([0], [0], color=COL_MED, lw=4),
]

ax1.set_prop_cycle('color', colors)

#weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax1);
if weekly_valid_pivot.shape[0] > 1:
    weekly_valid_pivot.iloc[-2].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax1);
ax1.plot(weekly_train_avg['Minuteofweek'], weekly_train_avg['Moulin'], color=COL_AVG, alpha=ALPHA);
ax1.plot(weekly_train_med['Minuteofweek'], weekly_train_med['Moulin'], color=COL_MED, alpha=ALPHA);


ax1.set_title("Seat availability: weekly view (rue du Fer à Moulin)", fontsize=18);

ax1.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax1.set_xticklabels(days_of_the_week.values());
ax1.set_xlabel("Day of week", fontsize=14);
ax1.tick_params(axis='x', labelrotation=90);

ax1.set_ylabel("Available seats", fontsize=14);
ax1.set_yticks(range(0, 80, 10));

# ax1.get_legend().remove();
# ax1.legend(['])
ax1.legend(custom_lines, ['training', 'validation', 'median'], loc='upper left');
ax1.grid();

ax2.set_prop_cycle('color', colors)

# weekly_train_pivot.transpose().plot(style='-', linewidth=5, color=COL_TRAIN, alpha=0.2, ax=ax2);
weekly_valid_pivot.iloc[-1].plot(style='-', linewidth=2, color=COL_VALID, alpha=1, ax=ax2);
ax2.plot(weekly_train_avg['Minuteofweek'], weekly_train_avg['Moulin'], color=COL_AVG, alpha=ALPHA);
ax2.plot(weekly_train_med['Minuteofweek'], weekly_train_med['Moulin'], color=COL_AVG, alpha=ALPHA);


ax2.set_xticks([d*ONE_DAY*RESOL for d in range(0, 7)])
ax2.set_xticklabels(days_of_the_week.values());
ax2.set_xlabel("Day of week", fontsize=14);
ax2.tick_params(axis='x', labelrotation=90);

ax2.set_ylabel("Available seats", fontsize=14);
ax2.set_yticks(range(0, 80, 10));

ax2.legend(custom_lines, ['training', 'validation', 'average'], loc='upper right');
ax2.grid();

if SAVEFIGS:
    plt.savefig('moulin-weekly-timeseries.png')

# Visualization with heatmaps
With the pivot tables from above we can also generate an alternattive representation of the data using heatmaps.  

## Visualizing historical data with a heatmap
We first view the entire dataset with a heatmap:

In [ ]:
fig, ax = plt.subplots(figsize=(16, 20));

ax.set_facecolor("black")
mask = daily_pivot.sort_index(ascending=False).isnull()
sns.heatmap(daily_pivot.sort_index(ascending=False),
            cmap="Oranges_r",
            mask=mask,
            ax=ax);

ax.set_xlabel("Time of day", fontsize=14);
ax.set_xticks([h*ONE_HOUR for h in range(0, 24)]);
ax.set_xlabel("Time of day", fontsize=14);
ax.set_xticks([h*ONE_HOUR for h in range(0, 24)]);
ax.set_xticklabels(["{0:02d}:00".format(x//ONE_HOUR) for x in ax.get_xticks()])
ax.tick_params(axis='x', labelrotation=45);

ax.set_yticklabels([new_y_ticklabel(text) for text in ax.get_yticklabels()]);
ax.set_ylabel("Date", fontsize=14);
ax.tick_params(axis='y', labelrotation=0);

ax.set_title("Seat availability (rue du Fer à Moulin)", fontsize=18);
if SAVEFIGS:
    plt.savefig("moulin-availability-heatmap.png");

### Inspecting missing data

The black cells represent missing data.  

Some of these missing data were created while generating the pivot table.  Indeed, data was missing at the beginning of the first day of data collection, and data is not yet available for the remainder of the last day of data collection.  Let's set these values to `-1`:

In [ ]:
# Missing data at beginning of first day
daily_pivot.iloc[0].fillna(value=-1, inplace=True)  
# Missing data at end of last day
daily_pivot.iloc[-1].fillna(method='ffill', inplace=True)

In addition, there are missing data on Sun 31 Apr 2019.  These are actually due to the switch to daylight saving time in France, which occurs at 2am.  Let's find them in the pivot table:

In [ ]:
nan_rows = np.where(daily_pivot.apply(axis=1, func=lambda x : any(pd.isnull(x))))
nan_cols = np.where(daily_pivot.apply(axis=0, func=lambda x : any(pd.isnull(x))))
len(nan_rows[0]), len(nan_cols[0])

There remains `1` row with `30` `nan` values:

In [ ]:
daily_pivot.iloc[nan_rows[0], nan_cols[0]]

Indeed, these occur between 2am and 3am (between minute `2*60 = 120` and `3*60 = 180`).

In [ ]:
daily_pivot.fillna(value=-1, inplace=True)
daily_pivot = daily_pivot.astype(np.int8)

## Visualizing averages with a heatmap
To represent the average data, we calculate another pivot table (alternatively, we could have added a bit more to `weekly_avg` above).

In [ ]:
train_avg = train_data.groupby(['Dayofweek', 'Minuteofday']).apply(lambda x: x['Moulin'].mean().round().astype(np.uint8)).reset_index()
train_avg.rename(columns={0: 'Moulin'}, inplace=True)

In [ ]:
train_avg_pivot = pd.pivot_table(train_avg, index='Dayofweek', columns='Minuteofday', values='Moulin')
train_avg_pivot.fillna(train_avg_pivot.mean(axis=0), inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE);

sns.heatmap(train_avg_pivot.sort_index(ascending=False),
          # annot=True,
            cmap="Oranges_r",
            # linewidths=.5, fmt='.3g',
            ax=ax);
ax.set_xlabel("Time of day", fontsize=14);
ax.set_xticks([h*ONE_HOUR for h in range(0, 24)]);
ax.set_xticklabels(["{0:02d}:00".format(x//ONE_HOUR) for x in ax.get_xticks()])
ax.tick_params(axis='x', labelrotation=45);
ax.set_yticklabels(days_of_the_week.values());
ax.set_title("Average seat availability (rue du Fer à Moulin)", fontsize=18);
ax.set_ylabel("Day of week", fontsize=14);
ax.tick_params(axis='y', labelrotation=0);
if SAVEFIGS:
    plt.savefig('moulin-average-heatmap.png')